In [1]:
import sys
sys.version

'3.7.2 (default, Dec 29 2018, 06:19:36) \n[GCC 7.3.0]'

In [2]:
import torch
print(torch.__version__)

1.0.0


In [3]:
#It's important that you have a working NVidia GPU set up. 
#The programming framework used to behind the scenes to work with NVidia GPUs is called CUDA. 
#Therefore, you need to ensure the following line returns True before you proceed. 
#If you have problems with this, please check the FAQ and ask for help on the forums.


In [4]:
#import tensorflow as tf
#tf.test.gpu_device_name()
torch.cuda.is_available()

False

In [5]:
from fastai.learner import *
from fastai.column_data import *

ModuleNotFoundError: No module named 'fastai.learner'

In [ ]:
path='../../data/kkbox_music/'
!ls '../../data/kkbox_music/'

In [ ]:
songs = pd.read_csv(path+'songs.csv')
songs.head()

In [ ]:
songs.shape

In [ ]:
songs.info()

In [ ]:
# If there are recurring listening event(s) triggered within a month after the user’s very first
# observable listening event, its target is marked 1, and 0 otherwise in the training set.
train = pd.read_csv(path+'train.csv')
train.head()

In [ ]:
train.shape

In [ ]:
members = pd.read_csv(path+'members.csv')
members.head()

In [ ]:
members.shape

In [ ]:
song_extra_info = pd.read_csv(path+'song_extra_info.csv')
song_extra_info.head()

In [ ]:
song_extra_info.shape

In [ ]:
test = pd.read_csv(path+'test.csv')
test.head()

In [ ]:
test.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Here follows the distribution of the members ages. As you can see there are several with age set to 0. 
#That is obsviously something we'll have to manage.

# bd = age
f,axarray = plt.subplots(1,1,figsize=(15,10))
agehist = members.groupby(['bd'],as_index=False).count()
sns.barplot(x=agehist['bd'],y=agehist['gender'])

In [ ]:
#Members come from 21 different cities identified with an integer index ranging from 1 to 21. 
#Here is the distribution of the members cities.
f,axarray = plt.subplots(1,1,figsize=(15,10))
cityhist = members.groupby(['city'],as_index=False).count()
sns.barplot(x=cityhist['city'],y=cityhist['gender'])

In [ ]:
## Songs

print ("There are:",len(songs),"songs;",len(songs['composer'].unique()),
       "composers for",len(songs['genre_ids'].unique()),"genres in",
      len(songs['language'].unique()),"languages")

In [ ]:
#The following returns summarized aggregate information to each table accross each field.
display(DataFrameSummary(train).summary())

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train['msno'].value_counts()

In [ ]:
train.hist(bins=50, figsize=(10,5))

In [ ]:
display(DataFrameSummary(songs).summary())

In [ ]:
display(DataFrameSummary(members).summary())

# Reducing Train data set for quick evaluation
## from 7.3 mln to 30K

In [ ]:
smallTrain = train.head(n=30000)
smallTrain.to_csv(path+'small_train.csv',encoding='utf-8')
train.shape, smallTrain.shape

In [ ]:
#create a crosstab
g=smallTrain.groupby('msno')['target'].count()
topUsers=g.sort_values(ascending=False)[:15]

g=smallTrain.groupby('song_id')['target'].count()
topMovies=g.sort_values(ascending=False)[:15]

top_r = smallTrain.join(topUsers, rsuffix='_r', how='inner', on='msno')
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='song_id')

pd.crosstab(top_r.msno, top_r.song_id, top_r.target, aggfunc=np.sum)

# Fast.ai Collaborative filtering

In [ ]:
#??CollabFilterDataset

In [ ]:
val_idxs = get_cv_idxs(len(smallTrain))
wd=2e-4
n_factors = 50

In [ ]:
cf = CollabFilterDataset.from_csv(path, 'small_train.csv', 'msno', 'song_id', 'target')
learn = cf.get_learner(n_factors, val_idxs, 64, opt_fn=optim.Adam)

In [ ]:
# loss in plain MSE
learn.fit(1e-2, 2, wds=wd, cycle_len=1, cycle_mult=2)

In [ ]:
preds = learn.predict()
preds

In [ ]:
y=learn.data.val_y
sns.jointplot(preds, y, kind='hex', stat_func=None);

## Create the complete dataset by merging train, members and songs dataframes

In [ ]:
FullDF = train.merge(members,how='inner',on='msno')
FullDF.head()

In [ ]:
FullDF = FullDF.merge(songs,how='inner',on='song_id')
FullDF.head()

In [ ]:
train.shape, FullDF.shape

In [ ]:
FullDF.info()

In [ ]:
display(DataFrameSummary(FullDF).summary())

In [ ]:
smallFullDF = FullDF.head(n=50000)
smallFullDF.to_csv(path+'smallFullDF.csv',encoding='utf-8')

In [ ]:
val_idxs = get_cv_idxs(len(smallFullDF))
wd=2e-4
n_factors = 50

In [ ]:
# !!! this doesn't work !!!
#cff = CollabFilterDataset.from_csv(path, 'smallFullDF.csv', ['msno','city','bd','gender'], ['song_id','song_length','genre_ids'], 'target')
#learn2 = cff.get_learner(n_factors, val_idxs, 64, opt_fn=optim.Adam)
# loss in plain MSE
#learn2.fit(1e-2, 2, wds=wd, cycle_len=1, cycle_mult=2)

# example using fast.ai ColumnarModelData
## https://towardsdatascience.com/structured-deep-learning-b8ca4138b848

Next we'll fill in missing values to avoid complications with NA's. NA (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary signal value that doesn't otherwise appear in the data.

In [ ]:
smallFullDF.genre_ids.head()

In [ ]:
smallFullDF['city'].value_counts()

In [ ]:
# for col in smallFullDF.columns:
#     if smallFullDF[col].dtype == object:
#         #print(FullDF[col])
#         smallFullDF[col] = smallFullDF[col].astype('category')
#         #FullDF[col] = le.fit_transform(FullDF[col].astype(str))
#         #FullDF[col] = le.fit_transform(FullDF[col])
#     elif smallFullDF[col].dtype == bool:
#         smallFullDF[col] = smallFullDF[col].astype('category')
#     else:
#         pass

In [ ]:
 type(smallFullDF['msno'])

In [ ]:
smallFullDF.info()

In [ ]:
smallFullDF.T

In [ ]:
# fill in unknown NaN values for selected columns
Fillvalues = {'msno': 'missing',
              'song_id': 'missing',
              'genre_ids': 'missing', 
              'artist_name': 'missing', 
              'bd': 0,
              'city':99,
              'song_length':1,
              'gender': 'missing'}
smallFullDF.fillna(value=Fillvalues, inplace=True)

In [ ]:
smallFullDF['bd'].value_counts()

In [ ]:
# simplify table be removing unused columns (normally wouldn't do that and use all)
#better option is to do df_small = df2[['id', 'budget', ...]].copy()
smallFullDF.drop('source_system_tab',1,inplace=True)
smallFullDF.drop('source_screen_name',1,inplace=True)
smallFullDF.drop('source_type',1,inplace=True)
smallFullDF.drop('registered_via',1,inplace=True)
smallFullDF.drop('registration_init_time',1,inplace=True)
smallFullDF.drop('expiration_date',1,inplace=True)
smallFullDF.drop('composer',1,inplace=True)
smallFullDF.drop('lyricist',1,inplace=True)
smallFullDF.drop('language',1,inplace=True)

In [ ]:
smallFullDF.T

In [ ]:
smallFullDF.info()

In [ ]:
#select which vars are continueouis and categorical, and dependant variable 
cat_vars = ['msno', 'song_id', 'city', 'gender', 'genre_ids', 'artist_name']
contin_vars = ['song_length']
dep = 'target'

In [ ]:
joined = smallFullDF[cat_vars+contin_vars+[dep]].copy()

In [ ]:
joined

In [ ]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [ ]:
joined.info()

In [ ]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')

In [ ]:
joined.info()

In [ ]:
n = len(joined); n


In [ ]:
idxs = get_cv_idxs(n, val_pct=50000/n)
joined_samp = joined.iloc[idxs].set_index("msno")
samp_size = len(joined_samp); samp_size

In [ ]:
# To run on the full dataset, use this instead:
#samp_size = n
#joined_samp = joined.set_index("msno")

In [ ]:
joined_samp.head(2)

In [ ]:
from fastai.structured import *
from fastai.column_data import *

In [ ]:
df, y, nas, mapper = proc_df(joined_samp, 'target', do_scale=True)
yl = np.log(y)

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))
print(train_size, len(df))
print(val_idx)

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [ ]:


def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)



In [ ]:
#We can create a ModelData object directly from out data frame.
md = ColumnarModelData.from_data_frame(path, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128)